In [39]:
import time
import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format ='retina'
import random
from functools import reduce
import spotipy
from typing import List
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy import oauth2
from os import listdir
import ast


In [40]:
cid = '84a918187a0d4fc5a66dd8e062fff899'
secret = '9f4fea70be164dd7bb5ffcf653b33d29'
redirect_uri='http://localhost:8080/callback/'
username = 'egtrwnupl4nlhpsxae4aecqno'

In [41]:
scope = 'user-top-read playlist-modify-private playlist-modify-public'
token = util.prompt_for_user_token(username, scope, client_id=cid, client_secret=secret, redirect_uri=redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [42]:
def fetch_audio_features(sp, df):
    playlist = df[['track_id','track_name']] 
    index = 0
    audio_features = []
    
    # Make the API request
    while index < playlist.shape[0]:
        audio_features += sp.audio_features(playlist.iloc[index:index + 50, 0])
        index += 50
    
    # Create an empty list to feed in different charactieritcs of the tracks
    features_list = []
    #Create keys-values of empty lists inside nested dictionary for album
    for features in audio_features:
        features_list.append([features['danceability'],
                              features['acousticness'],
                              features['energy'], 
                              features['tempo'],
                              features['instrumentalness'], 
                              features['loudness'],
                              features['liveness'],
                              features['duration_ms'],
                              features['key'],
                              features['valence'],
                              features['speechiness'],
                              features['mode']
                             ])
    
    df_audio_features = pd.DataFrame(features_list, columns=['danceability', 'acousticness', 'energy','tempo', 
                                                             'instrumentalness', 'loudness', 'liveness','duration_ms', 'key',
                                                             'valence', 'speechiness', 'mode'])
    
    # Create the final df, using the 'track_id' as index for future reference
    df_playlist_audio_features = pd.concat([playlist, df_audio_features], axis=1)
    df_playlist_audio_features.set_index('track_name', inplace=True, drop=True)
    return df_playlist_audio_features

# Getting the songs from all of Spotify's playlist

In [43]:
 # Getting playlist IDs from each of Spotify's playlists
playlists = sp.user_playlists('spotify')
spotify_playlist_ids = []
 while playlists:
        for i, playlist in enumerate(playlists['items']):
            spotify_playlist_ids.append(playlist['uri'][-22:])
        if playlists['next']:
             playlists = sp.next(playlists)
        else:
             playlists = None
spotify_playlist_ids[:20]

['37i9dQZF1DXcBWIGoYBM5M',
 '37i9dQZF1DX0XUsuxWHRQd',
 '37i9dQZF1DX1lVhptIYRda',
 '37i9dQZF1DX10zKzsJ2jva',
 '37i9dQZF1DX4JAvHpjipBk',
 '37i9dQZF1DX4sWSpwq3LiO',
 '37i9dQZF1DX4SBhb3fqCJd',
 '37i9dQZF1DWXRqgorJj26U',
 '37i9dQZF1DX4dyzvuaRJ0n',
 '37i9dQZF1DXcF6B6QPhFDv',
 '37i9dQZF1DWXJfnUiYjUKT',
 '37i9dQZF1DXcRXFNfZr7Tp',
 '37i9dQZF1DX4o1oenSJRJd',
 '37i9dQZF1DXbTxeAdrVG2l',
 '37i9dQZF1DX4UtSsGT1Sbe',
 '37i9dQZF1DWTJ7xPn4vNaz',
 '37i9dQZF1DXaKIA8E7WcJj',
 '37i9dQZF1DWSV3Tk4GO2fq',
 '37i9dQZF1DWTwnEm1IYyoj',
 '37i9dQZF1DX2A29LI7xHn1']

In [7]:
len(spotify_playlist_ids)

1147

## Getting tracks from spotify playlist

In [8]:
# Creating a function to get the first 50 tracks IDs from a playlist
def getTrackIDs(playlist_id):
    playlist = sp.user_playlist('spotify', playlist_id)
    for item in playlist['tracks']['items'][:50]:
        track = item['track']
        ids.append(track['id'])
    return

In [9]:
# Creating a function get features of each track from track id
def getTrackFeatures(track_id):
  meta = sp.track(track_id)
  features = sp.audio_features(track_id)

  # meta
  track_id = track_id
  name = meta['name']
  album = meta['album']['name']
  artist = meta['album']['artists'][0]['name']
  release_date = meta['album']['release_date']
  length = meta['duration_ms']
  popularity = meta['popularity']

  # features
  acousticness = features[0]['acousticness']
  danceability = features[0]['danceability']
  energy = features[0]['energy']
  instrumentalness = features[0]['instrumentalness']
  liveness = features[0]['liveness']
  loudness = features[0]['loudness']
  speechiness = features[0]['speechiness']
  tempo = features[0]['tempo']
  time_signature = features[0]['time_signature']

  track = [track_id, name, album, artist, release_date, length, popularity, danceability, acousticness, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature]
  return track

In [10]:
%%time
# # Gathering track ids
ids = []
for x in spotify_playlist_ids[:200]:
    getTrackIDs(x)
ids[:5]

TypeError: 'NoneType' object is not subscriptable

In [11]:
df = pd.read_csv('data/playlist_songs.csv')
df.head()

,track_id,name,album,artist,release_date,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,7MAibcTli4IisCtbHKrGMh,Leave The Door Open,Leave The Door Open,Bruno Mars,2021-03-05,242096,90,0.586,0.1820,0.616,0.000000,0.0927,-7.964,0.0324,148.088,4
1,5QO79kh1waicV47BqGRL3g,Save Your Tears,After Hours,The Weeknd,2020-03-20,215626,97,0.680,0.0212,0.826,0.000012,0.5430,-5.487,0.0309,118.051,4
2,1diS6nkxMQc3wwC4G1j0bh,We're Good,Future Nostalgia (The Moonlight Edition),Dua Lipa,2021-02-11,165506,88,0.722,0.0319,0.588,0.000000,0.1830,-5.932,0.0544,134.010,4
3,4u4NyuceXP7Uzh7XFJKCr1,Hold On,Hold On,Justin Bieber,2021-03-05,170813,89,0.658,0.0106,0.634,0.000000,0.1320,-5.797,0.0413,139.980,4
4,3Ofmpyhv5UAQ70mENzB277,Astronaut In The Ocean,Astronaut In The Ocean,Masked Wolf,2021-01-06,132780,94,0.778,0.1750,0.695,0.000000,0.1500,-6.865,0.0913,149.996,4


In [12]:
# Dropping columns that could lead to data leakage
df = df.drop(columns=['name', 'album', 'artist', 'release_date'])
df.head()

,track_id,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,7MAibcTli4IisCtbHKrGMh,242096,90,0.586,0.1820,0.616,0.000000,0.0927,-7.964,0.0324,148.088,4
1,5QO79kh1waicV47BqGRL3g,215626,97,0.680,0.0212,0.826,0.000012,0.5430,-5.487,0.0309,118.051,4
2,1diS6nkxMQc3wwC4G1j0bh,165506,88,0.722,0.0319,0.588,0.000000,0.1830,-5.932,0.0544,134.010,4
3,4u4NyuceXP7Uzh7XFJKCr1,170813,89,0.658,0.0106,0.634,0.000000,0.1320,-5.797,0.0413,139.980,4
4,3Ofmpyhv5UAQ70mENzB277,132780,94,0.778,0.1750,0.695,0.000000,0.1500,-6.865,0.0913,149.996,4


In [13]:
# Dropping duplicated songs
df = df.drop_duplicates(subset=['track_id'])
df['track_id'].value_counts()


7MAibcTli4IisCtbHKrGMh    1
7KT7VGnPU5QVXN3q1BOeqb    1
2rkCYgvzyHp1AESIlJcqqY    1
7EYLrneh08x29IAWLl7Tst    1
2x4i2WnE05D2Z7yEeytmPr    1
                         ..
2Xa0oBYSZSXNk4DtNyrcYf    1
48O6kz322Dzu1R6Al5147q    1
3MDvya4tKmSsyPW9oXygB4    1
4OsLDuaH0bWR6xM6nj66F8    1
0jo2BDAxfgNrL5VplFuulx    1
Name: track_id, Length: 8883, dtype: int64

## Getting user's favorite tracks

In [15]:
# Getting top 50 tracks from user
results = sp.current_user_top_tracks(limit=1000, offset=0,time_range='short_term')


In [16]:
# Convert it to Dataframe
track_name = []
track_id = []
artist = []
album = []
duration = []
popularity = []
for i, items in enumerate(results['items']):
        track_name.append(items['name'])
        track_id.append(items['id'])
        artist.append(items["artists"][0]["name"])
        duration.append(items["duration_ms"])
        album.append(items["album"]["name"])
        popularity.append(items["popularity"])

# Create the final df   
df_favourite = pd.DataFrame({ "track_name": track_name, 
                             "album": album, 
                             "track_id": track_id,
                             "artist": artist, 
                             "duration": duration, 
                             "popularity": popularity})

df_favourite.head()

,track_name,album,track_id,artist,duration,popularity
0,Escapism.,Escapism. / The Thrill Is Gone.,5Z2MiIZ5I3jJvvmeWMLbOQ,RAYE,272373,89
1,Kill Bill,SOS,3OHfY25tqY28d16oZczHc8,SZA,153946,92
2,Be Alright,Be Alright,3EPXxR3ImUwfayaurPi3cm,Dean Lewis,196373,75
3,People You Know,Rare,63mdJr3NMY3wReOkNE6c6W,Selena Gomez,194900,75
4,IDK You Yet,IDK You Yet (Live on Piano),0j1Ia2lQWrcXrQZI4AdJlk,Alexander 23,184638,73


In [17]:
%%time
# Getting track features for each song in favorite song dataframe
fav_tracks = []
for track in df_favourite['track_id']:
    try:  
        track = getTrackFeatures(track)
        fav_tracks.append(track)
    except:
        pass

CPU times: total: 78.1 ms
Wall time: 14.9 s


In [18]:
# Create favorite track wtih audio features dataset
df_fav = pd.DataFrame(fav_tracks, columns = ['track_id', 'name', 'album', 'artist', 'release_date', 'length', 'popularity', 'danceability', 'acousticness', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature'])
df_fav.head()

,track_id,name,album,artist,release_date,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,5Z2MiIZ5I3jJvvmeWMLbOQ,Escapism.,Escapism. / The Thrill Is Gone.,RAYE,2022-10-12,272373,89,0.538,0.1380,0.742,0.000047,0.0934,-5.355,0.1140,96.107,4
1,3OHfY25tqY28d16oZczHc8,Kill Bill,SOS,SZA,2022-12-09,153946,92,0.644,0.0543,0.728,0.169000,0.1610,-5.750,0.0351,88.993,4
2,3EPXxR3ImUwfayaurPi3cm,Be Alright,Be Alright,Dean Lewis,2018-06-29,196373,75,0.553,0.6970,0.586,0.000000,0.0813,-6.319,0.0362,126.684,4
3,63mdJr3NMY3wReOkNE6c6W,People You Know,Rare,Selena Gomez,2020-01-10,194900,75,0.695,0.0881,0.620,0.000054,0.1110,-8.082,0.0521,150.032,4
4,0j1Ia2lQWrcXrQZI4AdJlk,IDK You Yet,IDK You Yet (Live on Piano),Alexander 23,2020-06-23,184638,73,0.648,0.8860,0.308,0.077400,0.1240,-11.729,0.0400,76.997,1


In [19]:
# Dropping columns that could lead to data leakage
df_fav = df_fav.drop(columns=['name', 'album', 'artist', 'release_date'])
df_fav.head()

,track_id,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,5Z2MiIZ5I3jJvvmeWMLbOQ,272373,89,0.538,0.1380,0.742,0.000047,0.0934,-5.355,0.1140,96.107,4
1,3OHfY25tqY28d16oZczHc8,153946,92,0.644,0.0543,0.728,0.169000,0.1610,-5.750,0.0351,88.993,4
2,3EPXxR3ImUwfayaurPi3cm,196373,75,0.553,0.6970,0.586,0.000000,0.0813,-6.319,0.0362,126.684,4
3,63mdJr3NMY3wReOkNE6c6W,194900,75,0.695,0.0881,0.620,0.000054,0.1110,-8.082,0.0521,150.032,4
4,0j1Ia2lQWrcXrQZI4AdJlk,184638,73,0.648,0.8860,0.308,0.077400,0.1240,-11.729,0.0400,76.997,1


In [20]:
# Checking for duplicates in df_fav
df_fav['track_id'].value_counts()

5Z2MiIZ5I3jJvvmeWMLbOQ    1
0JXXNGljqupsJaZsgSbMZV    1
559SdaPI7mqy9IQiscuaMj    1
5kl6FcnZv8jQDgNSWNyfpN    1
6HU7h9RYOaPRFeh0R3UeAr    1
6I0KEvFfWszVJiCy5hum4V    1
6uvh0In7u1Xn4HgxOfAn8O    1
43zdsphuZLzwA9k4DJhU0I    1
4pTkDIlpFNoeWbahu8pVXW    1
09mEdoA6zrmBPgTEN5qXmN    1
2K7xn816oNHJZ0aVqdQsha    1
3BJe4B8zGnqEdQPMvfVjuS    1
3ZCTVFBt2Brf31RLEnCkWJ    1
3OHfY25tqY28d16oZczHc8    1
5K3SJuYEkvvrLbzOjPyRi1    1
6IPwKM3fUUzlElbvKw2sKl    1
6J2LdBN97cDWn0MLxYh9HB    1
7B3z0ySL9Rr0XvZEAjWZzM    1
0Ult84lvFuqNvbyXwyRQ58    1
1BCXUbnU0486n4eeTyyVIj    1
1MhXdlCQPnO56T57MfmaRm    1
1Qrg8KqiBpW07V7PNxwwwL    1
1TQXIltqoZ5XXyfCbAeSQQ    1
1dB1kzLOjTcmSHttRd8bnV    1
4JeOesjcWWlprTAvucDSJX    1
4IvuPZogXbY7LODs7qzr0W    1
4EDijkJdHBZZ0GwJ12iTAj    1
46u5B2WN4wryYLZuMAOmI4    1
3EPXxR3ImUwfayaurPi3cm    1
63mdJr3NMY3wReOkNE6c6W    1
0j1Ia2lQWrcXrQZI4AdJlk    1
3hRV0jL3vUpRrcy398teAU    1
7DbdUf8aHSYoliSjO6LZv6    1
2QjOHCTQ1Jl3zawyYOpxh6    1
4l0Mvzj72xxOpRrp6h8nHi    1
2tGvwE8GcFKwNdAXMnlb

In [21]:
# Creating favorite column to use in classification
df_fav['favorite'] = 1
df['favorite'] = 0 

In [22]:
# Checking if both datasets have the same columns
df.columns == df_fav.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

## Preparing dataset for model creation

In [23]:
print(df.shape)
print(df_fav.shape)

(8883, 13)
(50, 13)


In [24]:
# Combining the favorite dataframe with the dataframe of Spotify's songs
combined = pd.concat([df, df_fav])
combined.shape

(8933, 13)

In [25]:
combined.favorite.value_counts()

0    8883
1      50
Name: favorite, dtype: int64

In [26]:
# Creating dataframe of favorite songs
df_fav = combined.loc[combined['favorite'] == 1]
df_fav.head()

,track_id,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,favorite
0,5Z2MiIZ5I3jJvvmeWMLbOQ,272373,89,0.538,0.1380,0.742,0.000047,0.0934,-5.355,0.1140,96.107,4,1
1,3OHfY25tqY28d16oZczHc8,153946,92,0.644,0.0543,0.728,0.169000,0.1610,-5.750,0.0351,88.993,4,1
2,3EPXxR3ImUwfayaurPi3cm,196373,75,0.553,0.6970,0.586,0.000000,0.0813,-6.319,0.0362,126.684,4,1
3,63mdJr3NMY3wReOkNE6c6W,194900,75,0.695,0.0881,0.620,0.000054,0.1110,-8.082,0.0521,150.032,4,1
4,0j1Ia2lQWrcXrQZI4AdJlk,184638,73,0.648,0.8860,0.308,0.077400,0.1240,-11.729,0.0400,76.997,1,1


In [27]:
# Removing favorite songs from playlist songs
df = combined.loc[combined['favorite'] != 1]
df.shape

(8883, 13)

In [28]:
df_fav.shape

(50, 13)

In [29]:
# Saving these dataframes to use in model creation
df.to_csv('encoded_playlist_songs.csv', index=False)
df_fav.to_csv('favorite_songs.csv', index=False)